In [1]:
import pandas as pd

In [2]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()

    com = []
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            com.append(l.strip())
            count = 0

    return com

def seperate_comma(com_lst):
    raw_lst = [[ele if ele[-1] != ',' else ele[:-1] + ' ' + ',' for ele in command.split()] for command in com_lst]
    com_lst = [' '.join(ele) for ele in raw_lst]
    return com_lst

def read_all(num_files):
    com_lst = []
    for i in range(1,num_files+1):
        path = './Examples/EGPSR Examples_' + str(i)+ '.txt'
        # path = './Examples/EGPSR Examples.txt'
        com_lst.extend(read_command(path))

    com_lst = seperate_comma(com_lst)
    return com_lst

num_files = 10
# num_files = 1

com_lst = read_all(num_files)
# print(com_lst[3])

print('len: ',len(com_lst))
print(com_lst[:10])

len:  100000
['Could you find -B_find- Linda -B_find_per- at the entrance -B_find_dest- , take -B_guide- her -B_guide_per- to the exit -B_guide_dest- , say -B_tell- something -B_tell_what-', 'Go -B_go- to the living -i- room -B_go_dest- , find -B_find- Elizabeth -B_find_per- , and accompany -B_follow- her -B_follow_per- please', 'Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- white -I_tell_what- spoon are -I_tell_what- in -I_tell_what- the -I_tell_what- side -i- table -I_tell_what-', 'Please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- children -i- -I_tell_what- there -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- corridor -I_tell_what-', 'Locate -B_find- Elizabeth -B_find_per- at the end -i- table -B_find_dest- , pinpoint -B_find- a orange -B_find_obj- in the corridor -B_find_dest- , and pick -B_take- up -I_take- the fruits -B_take_obj- from the cupboard -B_take_sour- please', 'Find -B_find- the apple -B_find_obj- , locat

In [3]:
for i,s in enumerate(com_lst):
   if i == 1000:
       break
   if 'me -B_tell_dest- how -B_tell_what-' in s:
       print(s)

Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- white -I_tell_what- spoon are -I_tell_what- in -I_tell_what- the -I_tell_what- side -i- table -I_tell_what-
Please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- children -i- -I_tell_what- there -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- corridor -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- fruits -I_tell_what- items -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- counter -I_tell_what- please
Could you tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- people -i- are -i- sitting -I_tell_what- in -I_tell_what- the -I_tell_what- dining -i- room -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- men -i- are -i- raising -i- their -i- right -i- arm -I_tell_what- in -I_tell_what- the -I_tell_what- living -i- room -I_tell_what- please
Tell -B_tell- me -B_tell_dest- how -B_tell_wh

In [4]:
def find_sub_list(sub_list,this_list):
    start = this_list.index(sub_list[0])
    return (start,start+len(sub_list))

def process_person_pro(com_lst,path):
    lines = [line.strip() for line in open(path) if line.strip() != '' and line.strip() != 'it']
    pairs_lst = [(lines[i].split(' '),lines[i+1]) for i in range(0,len(lines)//2,2)] #[([B-referee, I-referee],referral),([B-referee, I-referee],referral),....]
    # print(pairs_lst)
    person_pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        lst = com.split()
        cur_referee  = pairs_lst[0][0]
        cur_referral = pairs_lst[0][1]

        words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
        str_referee = ' '.join(cur_referee).strip()
        if str_referee in words_lst and cur_referral in words_lst:
            # ------------ store command idx ------------
            person_pro_dict['idx'].append(i)

            # ------------ store original command  ------------
            person_pro_dict['words'].append(' '.join(words_lst))

            # ------------ store referral, the pronoun in word command ------------
            referral_idx = words_lst.index(cur_referral)
            person_pro_dict['referral'].append((referral_idx,cur_referral))

            # ------------ store referee, the word being referred ------------
            referee_idx = find_sub_list(cur_referee,words_lst)
            #print(referee_idx)
            person_pro_dict['referee'].append((referee_idx,str_referee))

            pairs_lst.pop(0)
            if len(pairs_lst) == 0:
                break
    return person_pro_dict

# path = './Examples/pros_test.txt'
path = './Examples/pros.txt'

person_pro_dict = process_person_pro(com_lst,path)
per_df = pd.DataFrame(person_pro_dict)
per_df

,idx,referee,referral,words
0,0,"((3, 4), Linda)","(9, her)","Could you find Linda at the entrance , take he..."
1,1,"((7, 8), Elizabeth)","(11, her)","Go to the living room , find Elizabeth , and a..."
2,5,"((5, 6), Charlie)","(13, him)","Find the apple , locate Charlie in the corrido..."
3,9,"((1, 2), John)","(9, him)","Contact John at the dining table , then greet him"
4,20,"((4, 5), Charlie)","(11, her)",Could you please contact Charlie at the bookca...
...,...,...,...,...
6410,50094,"((3, 4), Alex)","(10, him)","Robot please contact Alex at the bookcase , th..."
6411,50100,"((7, 8), Robin)","(11, him)","Go to the dining room , spot Robin , and lead ..."
6412,50108,"((12, 13), Francis)","(19, her)",Could you pick up the fruits from the end tabl...
6413,50112,"((8, 9), Alex)","(12, him)","Set knife on the counter , look for Alex , and..."


In [5]:
#{'_178-it_', '_285-it_', '_226-it_', '_226-it_,', '_95-it_', '_286-it_', '_227-it_', '_179-its_'}
def get_it_s_lab(com_lst):
    pro_tag_set = set()
    for com in com_lst:
        lst = com.split()
        for ele in lst:
            if ele[0] == '_':
                pro_tag_set.add(ele)
    return pro_tag_set



#"referral" : The referring wildcard.
#"referee" : The wildcard being referred.
def store_it_s_coref(com_lst):
    clean_com_lst = com_lst.copy()
    pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    pro_tag_set = get_it_s_lab(com_lst)
    for i,com in enumerate(com_lst):

        # if i == 50:
        #     break

        lst = com.split()

        the_tag = None
        for tag in list(pro_tag_set):
            if tag in lst:
                the_tag = [ele for ele in lst if ele[0] == '_'][0]
                break

        if the_tag != None: # has pronoun in this command
            clean_com_lst[i] = ' '.join([ele for ele in lst if ele[0] != '_'])
            # ------------ store command idx ------------
            pro_dict['idx'].append(i)

            words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
            pro_dict['words'].append(' '.join(words_lst))
            # ------------ store referral, the pronoun in word command ------------
            if 'its' in the_tag:
                pro_loc = words_lst.index('its')
                pro_dict['referral'].append((pro_loc,'its'))
            elif 'it' in the_tag:
                pro_loc = words_lst.index('it')
                pro_dict['referral'].append((pro_loc,'it'))

            # ------------ store referee, the word being referred ------------
            raw_loc = lst.index(the_tag)
            search_lst = lst[:raw_loc][::-1]
            obj_tag_count = 0
            i_count = 0
            rev_referee = []
            # print(search_lst)
            for j,ele in enumerate(search_lst):
                if obj_tag_count == 2:
                    rev_referee.append(ele)
                    if search_lst[j] == '-i-':
                        rev_referee.pop()
                        i_count += 1
                        continue

                    elif i_count ==len(rev_referee): # unfinished storing referee
                        continue

                    else:
                        break

                if '_obj-' in ele:
                    obj_tag_count += 1

            referee_lst = rev_referee[::-1]
            referee_loc = find_sub_list(referee_lst,words_lst)
            str_referee = ' '.join(referee_lst).strip()
            pro_dict['referee'].append((referee_loc,str_referee))

    return pro_dict,clean_com_lst

# get_it_s_lab(com_lst)
pro_dict,clean_com_lst = store_it_s_coref(com_lst)
it_df =pd.DataFrame(pro_dict)
it_df

,idx,referee,referral,words
0,14,"((7, 8), apple)","(15, it)","Please find Skyler , pick up the apple from th..."
1,24,"((10, 11), orange)","(17, it)","Could you please look for a apple , grasp the ..."
2,31,"((3, 4), orange)","(10, it)",Please retrieve the orange from the end table ...
3,50,"((9, 10), fruits)","(17, it)","Place grape juice on the cupboard , take the f..."
4,53,"((10, 11), fruits)","(18, it)","Find John at the end table , pick up the fruit..."
...,...,...,...,...
11038,99917,"((2, 3), dish)","(10, its)",Set a dish on the sink and a fork on its left
11039,99932,"((5, 6), snacks)","(11, it)",Robot please pick up a snacks from the desk an...
11040,99975,"((2, 3), cup)","(11, it)",Set a cup on the end table and a fork on it
11041,99991,"((2, 3), dish)","(10, its)",Put a dish on the bookcase and a spoon on its ...


In [6]:
def merge_df(com_lst,it_df,per_df):
    it_idx = it_df['idx'].tolist()
    per_idx = per_df['idx'].tolist()
    dic = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        if i not in it_idx and i not in per_idx:
            dic['idx'].append(i)
            dic['referee'].append('')
            dic['referral'].append('')
            words_lst = [ele for ele in com.split() if ele[0] != '_' and ele[0] != '-']
            dic['words'].append(' '.join(words_lst))

    rest_df = pd.DataFrame(dic)
    df = it_df.copy().append(per_df)
    df = df.append(rest_df)
    df = df.sort_values('idx')
    return df

pro_df = merge_df(clean_com_lst,it_df,per_df)
pro_df

/tmp/ipykernel_20828/3520584853.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = it_df.copy().append(per_df)
/tmp/ipykernel_20828/3520584853.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rest_df)


,idx,referee,referral,words
0,0,"((3, 4), Linda)","(9, her)","Could you find Linda at the entrance , take he..."
1,1,"((7, 8), Elizabeth)","(11, her)","Go to the living room , find Elizabeth , and a..."
0,2,,,Tell me how many white spoon are in the side t...
1,3,,,Please tell me how many children there are in ...
2,4,,,"Locate Elizabeth at the end table , pinpoint a..."
...,...,...,...,...
82538,99995,,,Tell me how many cleaning stuff items are in t...
82539,99996,,,Handshake the person wearing a hat in the livi...
82540,99997,,,Describe the person lying down at the exit to ...
11042,99998,"((2, 3), dish)","(10, it)",Place a dish on the cupboard and a spoon in it...


In [7]:
def gen_pro_labels(pro_df):
    referee_lst = pro_df['referee'].tolist()
    referral_lst = pro_df['referral'].tolist()
    words_lst = [ele.split() for ele in pro_df['words'].tolist()]
    pro_labels = [['O']*len(lst) for lst in words_lst]
    for sample_idx in range(len(words_lst)):
        if referral_lst[sample_idx] != '':
            referral_idx = referral_lst[sample_idx][0]
            pro_labels[sample_idx][referral_idx] = 'referral'

            referee_start = referee_lst[sample_idx][0][0]
            referee_end = referee_lst[sample_idx][0][1]
            pro_labels[sample_idx][referee_start] = 'B-referee'
            length = referee_end - referee_start
            if length > 1:
                pro_labels[sample_idx][referee_start + 1 : referee_end] = ['I-referee']*(length - 1)

    return pro_labels
pro_labels = gen_pro_labels(pro_df)
print(pro_labels[:10])

[['O', 'O', 'O', 'B-referee', 'O', 'O', 'O', 'O', 'O', 'referral', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-referee', 'O', 'O', 'O', 'referral', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-referee', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'referral'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-referee', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'referral']]


In [8]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):
        # print(i)
        # print(com)
        com = com.split()
        if '-i-' not in com:
            res.append(com)
            continue

        #deal with -i-
        elif '-i-' in com:
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')

            while '-i- -' in string:
                string = string.replace('-i- -','-')

            com = string.split()

            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0:
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])

            if i == 989:
                print(com)
                print(idx_lst)

            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                #com[lst[0]] = end_label.replace(',','')
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I')#.replace(',','')

                for j,idx in enumerate(lst):
                    if j == 0:
                        com[idx] = end_label ################################################
                    else:
                        label = com[lst[-1]+2]

                        com[idx] = label

            labelled = com.copy()

        res.append(labelled)
    return res

res = improve_labels(clean_com_lst)
# ' '.join(res[3])
res[0] #989

['Could',
 'you',
 'find',
 '-B_find-',
 'Linda',
 '-B_find_per-',
 'at',
 'the',
 'entrance',
 '-B_find_dest-',
 ',',
 'take',
 '-B_guide-',
 'her',
 '-B_guide_per-',
 'to',
 'the',
 'exit',
 '-B_guide_dest-',
 ',',
 'say',
 '-B_tell-',
 'something',
 '-B_tell_what-']

In [9]:
def generate_seq_out(res):
    seq_in = []
    seq_out = []
    for i,com in enumerate(res):
        in_lst = []
        clean_out_lst = []
        for ele in com:
#             print('ele; ',ele)
            if ele[0] != '-':
                in_lst.append(ele)
                clean_out_lst.append('O')
            else:
                clean_out_lst[-1] = ele.replace('-','',2)
                if ',' in ele:
                    clean_out_lst[-1] = clean_out_lst[-1].replace(',','')

        seq_in.append(in_lst)
        seq_out.append(clean_out_lst)

    return seq_in, seq_out

seq_in, seq_out = generate_seq_out(res)
print(seq_in[0])
seq_out[0]

['Could', 'you', 'find', 'Linda', 'at', 'the', 'entrance', ',', 'take', 'her', 'to', 'the', 'exit', ',', 'say', 'something']


['O',
 'O',
 'B_find',
 'B_find_per',
 'O',
 'O',
 'B_find_dest',
 'O',
 'B_guide',
 'B_guide_per',
 'O',
 'O',
 'B_guide_dest',
 'O',
 'B_tell',
 'B_tell_what']

# Check tell commands

In [10]:
for i,(com,lab) in enumerate(zip(seq_in,seq_out)):
    if 'B_tell' in lab:
        print(i)
        print(com)
        print(lab)
        print('-----------------------------')
    if i == 1000:
        break


0
['Could', 'you', 'find', 'Linda', 'at', 'the', 'entrance', ',', 'take', 'her', 'to', 'the', 'exit', ',', 'say', 'something']
['O', 'O', 'B_find', 'B_find_per', 'O', 'O', 'B_find_dest', 'O', 'B_guide', 'B_guide_per', 'O', 'O', 'B_guide_dest', 'O', 'B_tell', 'B_tell_what']
-----------------------------
2
['Tell', 'me', 'how', 'many', 'white', 'spoon', 'are', 'in', 'the', 'side', 'table']
['B_tell', 'B_tell_dest', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'O', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what']
-----------------------------
3
['Please', 'tell', 'me', 'how', 'many', 'children', 'there', 'are', 'in', 'the', 'corridor']
['O', 'B_tell', 'B_tell_dest', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what']
-----------------------------
10
['Robot', 'please', 'describe', 'the', 'person', 'standing', 'at', 'the', 'dishwasher', 'to', 'me']
['O', 'O', 'B_tell', 'O', 'B_tell_what', 'I_tell_

In [11]:
# need to process the pro df as well
def rem_say(seq_in,seq_out,pro_labels):
    say_idx = [i for i,labels in enumerate(seq_out) if 'B_say' in labels]
    seq_in = [seq for i,seq in enumerate(seq_in) if i not in  say_idx]
    seq_out = [seq for i,seq in enumerate(seq_out) if i not in  say_idx]
    seq_pro = [seq for i,seq in enumerate(pro_labels) if i not in  say_idx]
    return seq_in, seq_out, seq_pro

def rem_dups(com_lst, lab_lst, pro_labels,rem = True):
    com_lst = [' '.join(lst) for lst in com_lst]
    lab_lst = [' '.join(lst) for lst in lab_lst]
    if rem:
        com_lst, lab_lst, seq_pro = rem_say(com_lst,lab_lst,pro_labels)
    else:
        seq_pro = pro_labels

    df = pd.DataFrame (com_lst, columns = ['command'])
    df['label'] = lab_lst
    df['pronouns'] = seq_pro
    df2 = df.drop_duplicates(subset=["command"], keep='first')
    return df2

df = rem_dups(seq_in,seq_out,pro_labels, rem = False)
df

,command,label,pronouns
0,"Could you find Linda at the entrance , take he...",O O B_find B_find_per O O B_find_dest O B_guid...,"[O, O, O, B-referee, O, O, O, O, O, referral, ..."
1,"Go to the living room , find Elizabeth , and a...",B_go O O B_go_dest I_go_dest O B_find B_find_p...,"[O, O, O, O, O, O, O, B-referee, O, O, O, refe..."
2,Tell me how many white spoon are in the side t...,B_tell B_tell_dest B_tell_what I_tell_what I_t...,"[O, O, O, O, O, O, O, O, O, O, O]"
3,Please tell me how many children there are in ...,O B_tell B_tell_dest B_tell_what I_tell_what I...,"[O, O, O, O, O, O, O, O, O, O, O]"
4,"Locate Elizabeth at the end table , pinpoint a...",B_find B_find_per O O B_find_dest I_find_dest ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
99993,"Pinpoint Francis in the kitchen , spot Robin ,...",B_find B_find_per O O B_find_dest O B_find B_f...,"[O, O, O, O, O, O, O, O, O, O, O, O]"
99994,Robot please take the smallest object from the...,O O B_take O B_take_obj I_take_obj O O B_take_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
99996,Handshake the person wearing a hat in the livi...,B_greet O B_greet_per I_greet_per I_greet_per ...,"[O, O, O, O, O, O, O, O, O, O]"
99997,Describe the person lying down at the exit to ...,B_tell O B_tell_what I_tell_what I_tell_what I...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [12]:
def write_unique_intent(df):
    label = df['label'].tolist()
    label_lst = [l.split() for l in label]
    # unique_labels
    intents = [ele[2:]  for lst in label_lst for ele in lst  if ele.count('_')==1] #ele[2:] to remove the B_, I_
    #intents.extend(['PAD', 'O'])
    unique_intent_verb = [*set(intents)]
    unique_intent_B = ['B-' + intent for intent in unique_intent_verb]
    unique_intent_I = ['I-' + intent for intent in unique_intent_verb]

    # Interleave
    unique_intent = unique_intent_B + unique_intent_I
    unique_intent[::2] = unique_intent_B
    unique_intent[1::2] = unique_intent_I

    unique_intent = ['PAD','O'] + unique_intent

    with open('intent_label.txt','w') as f:
        for intent in unique_intent:
            f.write(intent)
            f.write('\n')
    return unique_intent

unique_intent = write_unique_intent(df)
unique_intent

['PAD',
 'O',
 'B-answer',
 'I-answer',
 'B-know',
 'I-know',
 'B-tell',
 'I-tell',
 'B-guide',
 'I-guide',
 'B-find',
 'I-find',
 'B-follow',
 'I-follow',
 'B-take',
 'I-take',
 'B-greet',
 'I-greet',
 'B-go',
 'I-go']

# Train Dev Test split

In [13]:
from sklearn.model_selection import train_test_split

def split(df):
    df_train, df_test_dev = train_test_split(df, test_size=0.25, random_state=42)
    df_dev, df_test = train_test_split(df_test_dev, test_size=0.8, random_state=42)

    env_dict = {'train' : df_train,
               'dev' : df_dev,
               'test' : df_test}
    for df in env_dict.values():
        df.columns = ['seq_in', 'seq_out', 'seq_pro']
    return env_dict

env_dict = split(df)
for env,df in env_dict.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out,seq_pro
8619,"Navigate to the living room , find the paprika...",B_go O O B_go_dest I_go_dest O B_find O B_find...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
83502,Tell me how many children are wearing T-shirts...,B_tell B_tell_dest B_tell_what I_tell_what I_t...,"[O, O, O, O, O, O, O, O, O, O, O, O]"
37877,Could you tell me how many children in the liv...,O O B_tell B_tell_dest B_tell_what I_tell_what...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
95143,Bring the sponge to the woman dressed in an or...,B_take O B_take_obj O O B_take_per I_take_per ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
5829,"Robot please look for Robert , pinpoint a choc...",O O B_find I_find B_find_per O B_find O B_find...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
86228,Bring the dish to the person dressed in an ora...,B_take O B_take_obj O O B_take_per I_take_per ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
56260,Could you tell me how many women in the corrid...,O O B_tell B_tell_dest B_tell_what I_tell_what...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
889,Give me the cleaning stuff at the left cupboard,B_take B_take_dest O B_take_obj I_take_obj O O...,"[O, O, O, O, O, O, O, O, O]"
20087,Bring the knife to the woman raising their rig...,B_take O B_take_obj O O B_take_per I_take_per ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_pro
17500,Could you deliver to me some fruits from the b...,O O B_take O B_take_dest O B_take_obj O O B_ta...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
94059,"Robot please find Robin at the couch , accompa...",O O B_find B_find_per O O B_find_dest O B_guid...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1786,"Place apple on the storage table , spot Alex i...",B_take B_take_obj O O B_take_dest I_take_dest ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
94263,"Pick up the orange from the cupboard , leave i...",B_take I_take O B_take_obj O O B_take_sour O B...,"[O, O, O, B-referee, O, O, O, O, O, referral, ..."
89924,Could you tell me how many children wearing sw...,O O B_tell B_tell_dest B_tell_what I_tell_what...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
66631,"Retrieve the apple , pinpoint Alex in the bedr...",B_take O B_take_obj O B_find B_find_per O O B_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
63119,"Spot the spoon , spot a orange in the corridor...",B_find O B_find_obj O B_find O B_find_obj O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
99832,"Take the apple , look for Linda in the dining ...",B_take O B_take_obj O B_find I_find B_find_per...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
16209,Could you please find a bowl in the living roo...,O O O B_find O B_find_obj O O B_find_dest I_fi...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_pro
4982,Pour some chocolate drink in a cup please,B_take O B_take_obj I_take_obj O O B_take_dest O,"[O, O, O, O, O, O, O, O]"
39912,"Robot please get the paprika from the desk , l...",O O B_take O B_take_obj O O B_take_sour O B_fi...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5676,Offer something to eat to all the people point...,B_take B_take_obj I_take_obj I_take_obj O B_ta...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
23575,"Could you find Alex , locate Robin at the end ...",O O B_find B_find_per O B_find B_find_per O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
84165,Could you bring me the food at the top dining ...,O O B_take B_take_dest O B_take_obj O O B_take...,"[O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
36458,Bring me some containers from the basket on th...,B_take B_take_dest O B_take_obj O O B_take_sou...,"[O, O, O, O, O, O, O, O, O, O, O]"
20888,Could you accompany the person dressed in an o...,O O B_follow O B_follow_per I_follow_per I_fol...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
59114,Please tell me how many men in the dining room...,O B_tell B_tell_dest B_tell_what I_tell_what I...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
23115,Tell me how many blue sponge are in the side t...,B_tell B_tell_dest B_tell_what I_tell_what I_t...,"[O, O, O, O, O, O, O, O, O, O, O]"


------------------------------------------------------------------


In [14]:
def find_sub_list(subl, l):
    # s_str = ' '.join(subl)
    # f_str = ' '.join(l)
    # num = f_str.count(s_str)
    # print(num)
    # ind_subl = []
    # for i in range(num):
    #     ind_subl.extend([i for i in range(len(l)) if l[i] in subl])
    ind_subl = [i for i in range(len(l)) if l[i] in subl]
    if len(ind_subl) == 1:
        return [ind_subl[0]]
    else:
        return ind_subl

l='O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what', 'I-what'
find_sub_list(['I-what', 'O', 'I-what'],l)


[0, 3, 4, 5, 6, 7, 8, 9, 10]

In [15]:
def seq_intent_out(com_lst,lab_lst):
    out_int_lst = []

    #polite = ['Please','Could you', 'Robot please', 'Could you please', 'please']
    polite_unique = ['Please','Could you', 'Robot', 'please']

    #sep = ['and', ',' , ', and']
    sep_unique = ['and', ',' ]
    for idx,(com,lab) in enumerate(zip(com_lst,lab_lst)):

        sentence = ' '.join(com)

        polite_dict = {key:find_sub_list(key.split(' '),com) for key in polite_unique if key in sentence}
        polite_idx = []
        for ele in polite_dict.values():
            polite_idx.extend(ele)

        sep_dict = {key:find_sub_list(key.split(' '),com) for key in sep_unique if key in sentence}
        sep_idx = []
        for ele in sep_dict.values():
            sep_idx.extend(ele)

        non_intent_idx = polite_idx + sep_idx
        lst = ['O' if i in non_intent_idx else None for i in range(len(lab))]

        if 'get to know' in ' '.join(com):
            get_to_idx = find_sub_list(['get','to'],com)
            for j in get_to_idx:
                lst[j] = 'O'

        if 'Get to know' in ' '.join(com):
            get_to_idx = find_sub_list(['Get','to'],com)
            for j in get_to_idx:
                lst[j] = 'O'

        if 'then' in com:
            j = com.index('then')
            lst[j] = 'O'


        intent_lst = []
        B = True
        for i,ele in enumerate(lst):
            if ele == None and B ==True:
                B = False
                if lab[i] != 'O':
                    intent = lab[i].split('_')[1]
                else:
                    intent = intent_lst[-1][0]
                intent_lst.append([intent,[i]])
            elif ele == None and B == False:
                intent_lst[-1][1].append(i)

            elif ele == 'O':
                B = True

        intent_seq = lst#[ele if ele.count('_') != 1 else ele.replace('_','-') for ele in lab]
        for instance_lst in intent_lst:
            # for subtask_lst in instance_lst:
            intent, idx_lst =  instance_lst[0], instance_lst[1]
            begin = idx_lst.pop(0)
            intent_seq[begin] = 'B-' + intent
            for i in idx_lst:
                intent_seq[i] = 'I-' + intent


        out_int_lst.append(intent_seq)

        # if idx == 8:
        if 'I_tell_what' in intent_seq:
            print('  '.join(com))
            print('lab: ', lab)
            print('polite_dict: ',polite_dict)
            print('sep_dict: ',sep_dict)
            print('lst: ', lst)
            print('intent_lst: ',intent_lst)
            print(intent_seq)
            print('-----------------------------------------------------------')
    return out_int_lst


#
lab_lst = [ele.split() for ele in env_dict['train']['seq_out'].tolist()]
com_lst = [ele.split() for ele in env_dict['train']['seq_in'].tolist()]
seq_intent_out(com_lst,lab_lst)

[['B-go',
  'I-go',
  'I-go',
  'I-go',
  'I-go',
  'O',
  'B-find',
  'I-find',
  'I-find',
  'O',
  'B-find',
  'I-find',
  'O'],
 ['B-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell'],
 ['O',
  'O',
  'B-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell'],
 ['B-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'O'],
 ['O',
  'O',
  'B-find',
  'I-find',
  'I-find',
  'O',
  'B-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'O',
  'O',
  'B-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find'],
 ['B-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'O',
  'B-take',
  'I-take',
  'I-take',
  'I-take

In [16]:
a = 'I-what O I-what'
b = a.replace('O','I-what')
a

'I-what O I-what'

In [17]:
def write(f_name,data_lst, space = True):
    with open(f_name, 'w') as f:
        for i,lst in enumerate(data_lst):
            if space == True:
                # print(i)
                # print(lst)
                com = ' '.join(lst)
            else:
                com = ''.join(lst)
            f.write(com)
            f.write('\n')
    return

def gen_files(env_name, data_df):
    # get seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]

    #get seq in
    com_lst = [ele.split() for ele in data_df['seq_in'].tolist()]
    #get pro labels lst
    pro_labels = data_df['seq_pro'].tolist()

    int_lst = seq_intent_out(com_lst,lab_lst)

    # get label
    label = []
    for lab_seq in int_lst:
        lst = []
        for lab in lab_seq:
            if lab == 'O':
                continue
            elif len(lst) == 0:
                lst.append(lab)
            elif lab not in lst:
                lst.append(lab)
        hash_taged = [ele+'#' if i != len(lst)-1 else ele for i,ele in enumerate(lst)]
        label.append(hash_taged)


    # get seq out
    # 1. remove the intent label out of the seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    lab_lst = [[ele if ele.count('_') == 2 else 'O' for ele in lst] for lst in lab_lst]
    # 2. remove the intentions names in the slot names
    slot_seq = []
    for ite,one_seq in enumerate(lab_lst):
        lst = []
        for slot in one_seq:
            if slot == 'O':
                lst.append('O')
            else:
                # with IBO
                slot_start = slot.find('_',2)+1
                entity_type = slot[slot_start:]
                lst.append(slot[0]+'-'+entity_type)


        temp = ' '.join(lst)
        if 'I-what O I-what' in temp:
            lst = temp.replace('I-what O I-what','I-what I-what I-what').split()

        slot_seq.append(lst)

    df = data_df.copy()
    df['label'] = label
    df['seq_intent'] = int_lst
    df['seq_out'] = slot_seq
    df['seq_pro'] = pro_labels


    # write all files
    write(f'./{env_name}/seq.in',com_lst)
    write(f'./{env_name}/seq.out',slot_seq)
    write(f'./{env_name}/seq_intent.out',int_lst)
    write(f'./{env_name}/seq_pro.out',pro_labels)
    write(f'./{env_name}/label',label, False)

    return df

df_train_all = gen_files('train',env_dict['train'])
df_dev_all = gen_files('dev',env_dict['dev'])
df_test_all = gen_files('test',env_dict['test'])

env_dict_all = {'train' : df_train_all,
               'dev' : df_dev_all,
               'test' : df_test_all}

for env,df in env_dict_all.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out,seq_pro,label,seq_intent
8619,"Navigate to the living room , find the paprika...","[O, O, O, B-dest, I-dest, O, O, O, B-obj, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-go#, I-go#, B-find#, I-find]","[B-go, I-go, I-go, I-go, I-go, O, B-find, I-fi..."
83502,Tell me how many children are wearing T-shirts...,"[O, B-dest, B-what, I-what, I-what, I-what, I-...","[O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[B-tell, I-tell, I-tell, I-tell, I-tell, I-tel..."
37877,Could you tell me how many children in the liv...,"[O, O, O, B-dest, B-what, I-what, I-what, I-wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, O, B-tell, I-tell, I-tell, I-tell, I-tell,..."
95143,Bring the sponge to the woman dressed in an or...,"[O, O, B-obj, O, O, B-per, I-per, I-per, I-per...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
5829,"Robot please look for Robert , pinpoint a choc...","[O, O, O, O, B-per, O, O, O, B-obj, I-obj, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find]","[O, O, B-find, I-find, I-find, O, B-find, I-fi..."
...,...,...,...,...,...
86228,Bring the dish to the person dressed in an ora...,"[O, O, B-obj, O, O, B-per, I-per, I-per, I-per...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
56260,Could you tell me how many women in the corrid...,"[O, O, O, B-dest, B-what, I-what, I-what, I-wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, O, B-tell, I-tell, I-tell, I-tell, I-tell,..."
889,Give me the cleaning stuff at the left cupboard,"[O, B-dest, O, B-obj, I-obj, O, O, B-sour, I-s...","[O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
20087,Bring the knife to the woman raising their rig...,"[O, O, B-obj, O, O, B-per, I-per, I-per, I-per...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_pro,label,seq_intent
17500,Could you deliver to me some fruits from the b...,"[O, O, O, O, B-dest, O, B-obj, O, O, B-sour, I...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
94059,"Robot please find Robin at the couch , accompa...","[O, O, O, B-per, O, O, B-dest, O, O, B-per, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-find#, I-find#, B-guide#, I-guide#, B-tell]","[O, O, B-find, I-find, I-find, I-find, I-find,..."
1786,"Place apple on the storage table , spot Alex i...","[O, B-obj, O, O, B-dest, I-dest, O, O, B-per, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
94263,"Pick up the orange from the cupboard , leave i...","[O, O, O, B-obj, O, O, B-sour, O, O, B-obj, O,...","[O, O, O, B-referee, O, O, O, O, O, referral, ...","[B-take#, I-take#, B-tell]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
89924,Could you tell me how many children wearing sw...,"[O, O, O, B-dest, B-what, I-what, I-what, I-wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, O, B-tell, I-tell, I-tell, I-tell, I-tell,..."
...,...,...,...,...,...
66631,"Retrieve the apple , pinpoint Alex in the bedr...","[O, O, B-obj, O, O, B-per, O, O, B-dest, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take#, B-find#, I-find#, B-follow#...","[B-take, I-take, I-take, O, B-find, I-find, I-..."
63119,"Spot the spoon , spot a orange in the corridor...","[O, O, B-obj, O, O, O, B-obj, O, O, B-dest, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-find#, I-find#, B-answer#, I-answer]","[B-find, I-find, I-find, O, B-find, I-find, I-..."
99832,"Take the apple , look for Linda in the dining ...","[O, O, B-obj, O, O, O, B-per, O, O, B-dest, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take#, B-find#, I-find#, B-follow#...","[B-take, I-take, I-take, O, B-find, I-find, I-..."
16209,Could you please find a bowl in the living roo...,"[O, O, O, O, O, B-obj, O, O, B-dest, I-dest, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find]","[O, O, O, B-find, I-find, I-find, I-find, I-fi..."


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_pro,label,seq_intent
4982,Pour some chocolate drink in a cup please,"[O, O, B-obj, I-obj, O, O, B-dest, O]","[O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
39912,"Robot please get the paprika from the desk , l...","[O, O, O, O, B-obj, O, O, B-sour, O, O, O, B-p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
5676,Offer something to eat to all the people point...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
23575,"Could you find Alex , locate Robin at the end ...","[O, O, O, B-per, O, O, B-per, O, O, B-dest, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find#, B-tell#, I-tell]","[O, O, B-find, I-find, O, B-find, I-find, I-fi..."
84165,Could you bring me the food at the top dining ...,"[O, O, O, B-dest, O, B-obj, O, O, B-sour, I-so...","[O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
...,...,...,...,...,...
36458,Bring me some containers from the basket on th...,"[O, B-dest, O, B-obj, O, O, B-sour, I-sour, I-...","[O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
20888,Could you accompany the person dressed in an o...,"[O, O, O, O, B-per, I-per, I-per, I-per, I-per...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-follow#, I-follow]","[O, O, B-follow, I-follow, I-follow, I-follow,..."
59114,Please tell me how many men in the dining room...,"[O, O, B-dest, B-what, I-what, I-what, I-what,...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, B-tell, I-tell, I-tell, I-tell, I-tell, I-..."
23115,Tell me how many blue sponge are in the side t...,"[O, B-dest, B-what, I-what, I-what, I-what, I-...","[O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[B-tell, I-tell, I-tell, I-tell, I-tell, I-tel..."


------------------------------------------------------------------


In [21]:
for i,slot in enumerate(env_dict_all['train']['seq_out'].tolist()):
    lab = ['O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what']
    lab1 = ['O', 'O', 'O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what']
    lab2 = ['O', 'O', 'O', 'B-obj', 'O', 'O', 'B-sour', 'O', 'O', 'O', 'B-obj', 'O', 'B-dest', 'O', 'O', 'B-dest', 'O', 'O', 'O', 'B-obj', 'I-obj']
    if ' '.join(slot) == ' '.join(lab) or ' '.join(slot) == ' '.join(lab2) :
        print(env_dict_all['train']['seq_in'].tolist()[i])


In [19]:
def write_unique_slot(env_dict_all):
    label = []
    for df in env_dict_all.values():
        label.extend(df['seq_out'].tolist())

    label_lst = label.copy()

    slots = [ele  for lst in label_lst for ele in lst if ele != 'O'] #ele[2:] to remove the B_, I_
    unique_slot = ['PAD','O'] + [*set(slots)]

    with open('slot_label.txt','w') as f:
        for slot in unique_slot:
            f.write(slot)
            f.write('\n')
    return unique_slot

unique_slot = write_unique_slot(env_dict_all)
unique_slot

['PAD',
 'O',
 'B-obj',
 'I-what',
 'B-what',
 'I-per',
 'B-sour',
 'I-sour',
 'I-obj',
 'B-dest',
 'I-dest',
 'B-per']

In [20]:
def show_unique_labels(env):
    # uni_slot = [line.strip() for line in open('slot_label.txt', 'r', encoding='utf-8')]
    # uni_intent = [line.strip() for line in open('intent_label.txt', 'r', encoding='utf-8')]

    slot_path =  f'./{env}/seq.out'
    intent_path = f'./{env}/seq_intent.out'

    slot_seq = [line.strip().split() for line in open(slot_path, 'r', encoding='utf-8')]
    intent_seq = [line.strip().split() for line in open(intent_path, 'r', encoding='utf-8')]

    slot_lst = [label for seq in slot_seq for label in seq]
    intent_lst = [label for seq in intent_seq for label in seq]

    env_uni_slot = [*set(slot_lst)]
    env_uni_intent = [*set(intent_lst)]
    print('env: ',env)
    print('Unique intent: ',env_uni_intent,len(env_uni_intent))
    print('Unique slots:  ',env_uni_slot,len(env_uni_slot),'\n')
    return


for env in env_dict_all.keys():
    show_unique_labels(env)

env:  train
Unique intent:  ['B-follow', 'B-answer', 'B-go', 'B-greet', 'I-go', 'B-know', 'I-answer', 'B-take', 'O', 'B-find', 'I-greet', 'I-take', 'I-find', 'B-guide', 'I-tell', 'I-guide', 'I-know', 'I-follow', 'B-tell'] 19
Unique slots:   ['B-obj', 'I-what', 'B-what', 'I-per', 'B-sour', 'I-sour', 'I-obj', 'B-dest', 'O', 'I-dest', 'B-per'] 11 

env:  dev
Unique intent:  ['B-follow', 'B-answer', 'B-go', 'B-greet', 'I-go', 'B-know', 'I-answer', 'B-take', 'O', 'B-find', 'I-greet', 'I-take', 'I-find', 'B-guide', 'I-tell', 'I-guide', 'I-know', 'I-follow', 'B-tell'] 19
Unique slots:   ['B-obj', 'B-per', 'B-sour', 'B-what', 'I-what', 'I-per', 'I-obj', 'B-dest', 'O', 'I-dest', 'I-sour'] 11 

env:  test
Unique intent:  ['B-follow', 'B-answer', 'B-go', 'B-greet', 'I-go', 'B-know', 'I-answer', 'B-take', 'O', 'B-find', 'I-greet', 'I-take', 'I-find', 'B-guide', 'I-tell', 'I-guide', 'I-know', 'I-follow', 'B-tell'] 19
Unique slots:   ['B-obj', 'B-per', 'B-sour', 'B-what', 'I-what', 'I-sour', 'I-per'